In [ ]:
import pandas as pd
teks = pd.read_excel("text.xlsx")

In [ ]:
teks

In [ ]:
teks.columns

In [ ]:
teks.isnull().sum()

In [ ]:
teks['Kalimat'].unique()

In [ ]:
teks['Label'].value_counts()

In [ ]:
teks['length_before_cleaning'] = teks['Text'].apply(lambda x: len(x))

In [ ]:
teks

In [ ]:
# Import library
import nltk #import library Natural Language Toolkit
from gensim.models import KeyedVectors
import numpy as np

In [ ]:
nltk.download('punkt')

In [ ]:
nltk.download('stopwords')

In [ ]:
#Lemmatisasi Text
from nltk.stem import WordNetLemmatizer
#tokenisasi text
from nltk.tokenize import word_tokenize

In [ ]:
# Download WordNet resource
nltk.download('wordnet')

In [ ]:
#reguler python (RegEx)
import re

In [ ]:
def convert_to_lower(text):
    return text.lower()
teks['Text'] = teks['Text'].apply(lambda x: convert_to_lower(x))

In [ ]:
def remove_numbers(text):
    number_pattern = r'\d+'
    without_number = re.sub(pattern=number_pattern, repl=" ", string=text)
    return without_number

teks['Text'] = teks['Text'].apply(lambda x: remove_numbers(x))

In [ ]:
import string

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

teks['Text'] = teks['Text'].apply(lambda x: remove_punctuation(x))

In [ ]:
from nltk.corpus import stopwords
def remove_stopwords(text):
    removed = []
    stop_words = list(stopwords.words("english"))
    tokens = word_tokenize(text)
    for i in range(len(tokens)):
        if tokens[i] not in stop_words:
            removed.append(tokens[i])
    return " ".join(removed)
teks['Text'] = teks['Text'].apply(lambda x: remove_punctuation(x))

In [ ]:
def remove_extra_white_spaces(text):
    single_char_pattern = r'\s+[a-zA-Z]\s+'
    without_sc = re.sub(pattern=single_char_pattern, repl=" ", string=text)
    return without_sc
teks['Text'] = teks['Text'].apply(lambda x: remove_extra_white_spaces(x))

In [ ]:
def lemmatizing(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    for i in range(len(tokens)):
        lemma_word = lemmatizer.lemmatize(tokens[i])
        tokens[i] = lemma_word
    return " ".join(tokens)
teks['Text'] = teks['Text'].apply(lambda x: lemmatizing(x))

In [ ]:
teks['length_after_cleaning'] = teks['Text'].apply(lambda x: len(x))

In [ ]:
teks

In [ ]:
teks.dtypes

In [ ]:
# Menghapus kolom 'Kota' dan mengubah dataframe asli
teks.drop('Kalimat', axis = 1, inplace=True)

In [ ]:
label_map = {
    'supports': 0,
    'attacks': 1,
}

teks['Label'] = teks['Label'].map(label_map)

In [ ]:
teks.head()

In [ ]:
teks_lstm = teks.copy()
teks_lstm

In [ ]:
# Pisahkan fitur dan label
X = teks['Text'] #fitur
y = teks['Label'] #target

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Inisialisasi vektorisasi teks (TF-IDF)
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)

**Dataset Balancing**

In [ ]:
from imblearn.over_sampling import ADASYN
ADA = ADASYN(random_state=130)
X_train_ADA, y_train_ADA = ADA.fit_resample(X_tfidf, y)

In [ ]:
# from imblearn.over_sampling import SMOTE
# # Inisialisasi SMOTE
# smote = SMOTE(random_state=42)

In [ ]:
from collections import Counter
print(Counter(y_train_ADA))

**SPLIT DATASET**

In [ ]:
from sklearn.model_selection import train_test_split
# Bagi dataset menjadi data pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X_train_ADA, y_train_ADA, test_size=0.3, random_state=42)

**NAIVE BAYES MULTINOMIAL**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

In [ ]:
# Inisialisasi model Naive Bayes Multinomial
NBM = MultinomialNB()
# Pelatihan model
NBM.fit(X_train, y_train)

In [ ]:
# Evaluasi model pada data pengujian
y_pred_nb = NBM.predict(X_test)

from sklearn.metrics import accuracy_score

print("Akurasi Naive Bayes Multinomial:", accuracy_score(y_test, y_pred_nb))

In [ ]:
print("Naive Bayes Multinomial Classification Report:\n", classification_report(y_test, y_pred_nb))

**SUPPORT VECTOR CLASSIFIER**

In [ ]:
from sklearn.svm import SVC

In [ ]:
# Inisialisasi model SVM
svc_model = SVC(kernel='linear')
# Pelatihan model
svc_model.fit(X_train, y_train)

In [ ]:
# Evaluasi model pada data pengujian
y_pred_svc = svc_model.predict(X_test)

from sklearn.metrics import accuracy_score

print("Akurasi SVC:", accuracy_score(y_test, y_pred_svc))

In [ ]:
print("SVC Classification Report:\n", classification_report(y_test, y_pred_svc))

**RANDOM FOREST**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Inisialisasi model Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
# Pelatihan model
rf_model.fit(X_train, y_train)

In [ ]:
# Evaluasi model pada data pengujian
y_pred_rf = rf_model.predict(X_test)

from sklearn.metrics import accuracy_score

print("Akurasi Random Forest:", accuracy_score(y_test, y_pred_rf))

In [ ]:
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_rf))

**K-Nearest Neighbors (KNN)**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
k = 3  # Jumlah tetangga yang akan digunakan
knn_classifier = KNeighborsClassifier(n_neighbors=k)
knn_classifier.fit(X_train, y_train)

In [ ]:
y_pred_knn = knn_classifier.predict(X_test)

# Mengukur kinerja model
print("Akurasi KNN:", accuracy_score(y_test, y_pred_knn))

In [ ]:
print("KNN Classification Report:\n", classification_report(y_test, y_pred_knn))

**XGBOOST**

In [ ]:
import xgboost as xgb

In [ ]:
xgb = xgb.XGBClassifier(
    objective="binary:logistic",  # Klasifikasi biner
    max_depth=3,                  # Kedalaman maksimum pohon
    learning_rate=0.1,            # Tingkat pembelajaran
    n_estimators=100              # Jumlah iterasi (pohon) atau estimator
)
xgb.fit(X_train, y_train)

In [ ]:
y_pred_xgb = xgb.predict(X_test)
# Mengukur kinerja model
print("Akurasi XGBoost:", accuracy_score(y_test, y_pred_xgb))

In [ ]:
print("XGBoost Classification Report:\n", classification_report(y_test, y_pred_xgb))

**LSTM (LONG SHORT TERM MEMORY) + TOKENIZER**

In [ ]:
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
# from sklearn.metrics import accuracy_score

In [ ]:
# teks_lstm = teks.copy()
# teks_lstm

In [ ]:
# # Pisahkan fitur dan label
# X = teks_lstm['Text'] #fitur
# y = teks_lstm['Label'] #target

In [ ]:
# # Tokenisasi
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(X)

# X_sequences = tokenizer.texts_to_sequences(X)

In [ ]:
# max_seq_length = 100
# X_padded = pad_sequences(X_sequences, maxlen=max_seq_length, padding='post', truncating='post')

In [ ]:
# # Balancing dataset menggunakan ADASYN
# ADASYN= ADASYN(random_state=42)
# X_ADA, y_ADA = ADASYN.fit_resample(X_padded, y)

In [ ]:
# # model LSTM
# model = Sequential()
# model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_seq_length))
# model.add(SpatialDropout1D(0.4))
# model.add(LSTM(196, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(2, activation='sigmoid'))

In [ ]:
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# # Melatih model pada seluruh dataset
# model.fit(X_ADA, y_ADA, epochs=10, batch_size=64, validation_data=(X_ADA, y_ADA))